# StanfordTPCAnalysis Tutorial

### Table of Contents:

1. [Getting Started](#gettingstarted) 
2. [How to import StanfordTPCAnalysis modules](#importmodules)
3. [Using the `StruckAnalysisConfiguration` class](#struckanalysisconfiguration)
4. [Accessing the reduced data](#accessing)
    1. [Looking at all the variables in the data tree](#allvariables)
5. [Analyzing the reduced data](#analyzing)
6. [Inspecting waveforms in the raw data](#inspectingwaveforms)


<a id="gettingstarted"></a>


# Getting started


The data pipeline from the Stanford TPC uses a piece of in-house software called `StanfordTPCAnalysis`, which processes the waveform-level data from the TPC and produced reduced data that can be more easily analyzed. This tutorial is intended to some examples on how to analyze the reduced data. 

#### StanfordTPCAnalysis

It's often useful, even after the data processing, to have access to some of the `StanfordTPCAnalysis` python modules. To get started, you'll need to install a few python packages that StanfordTPCAnalysis requires and also make analyzing the data a bit easier. The dependencies for StanfordTPCAnalysis are listed on the GitHub page:
https://github.com/bglenardo/StanfordTPCAnalysis
and should be easily installed using `pip install`. 

#### Pandas

To do the analysis, you'll definitely need `pandas`, as all of our data is stored in `pandas.DataFrame()` objects. Documentation on the `pandas` library is located on their website. Probably a good place to start is in their tutorials: 
https://pandas.pydata.org/docs/getting_started/intro_tutorials/02_read_write.html

#### Histlite

Histlite is a python package written by former nEXO-er Mike Richman. It provides an easy interface for making both 1-D and 2-D histograms, similar to what one can do with ROOT at the command line. You can find more info on the `readthedocs` web page: 
https://histlite.readthedocs.io/en/latest/
and can install the package using `pip install`. This package isn't a necessary dependency, but will be used in the tutorial below.



# Import relevant libraries


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cycler

import histlite as hl

import sys
sys.path.append('/g/g20/lenardo1/software/')
from StanfordTPCAnalysis.StruckAnalysisConfiguration import StruckAnalysisConfiguration

import os

plt.rcParams['axes.prop_cycle'] = cycler.cycler(color='bgrmyk')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

import time

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12



<a id="importmodules"></a>

# How to import StanfordTPCAnalysis modules?

The StanfordTPCAnalysis code is located on GitHub: https://github.com/bglenardo/StanfordTPCAnalysis

There are two ways to make sure your analysis code has access to the StanfordTPCAnalysis modules:
1. Append the directory where the StanfordTPCAnalysis folder is located to `sys.path`
2. Add the directory where the StanfordTPCAnalysis folder is located to your `PYTHONPATH` environment variable.

Since this notebook should be self-contained, we'll go with option 1 below:


In [20]:
#to import the custom modules you need to access the StanfordTPCAnalysis.* folder,
#the code of the modules can be found on the TMS github repository
import sys

# NOTE THAT THIS IS AN ABSOLUTE PATH -- it will need to be changed for each user
path_to_StanfordTPCAnalysis = '/g/g20/lenardo1/software'
sys.path.append( path_to_StanfordTPCAnalysis )


In [21]:
# StruckAnalysisConfiguration class organizes all the metadata from each run. 

from StanfordTPCAnalysis.StruckAnalysisConfiguration import StruckAnalysisConfiguration

# The Waveform class has some tools to draw waveforms for each event.

from StanfordTPCAnalysis.WaveformAnalysis import Waveform



<a id="struckanalysisconfiguration"></a>

# Using the StruckAnalysisConfiguration class

The `StruckAnalysisConfiguration` class organizes all the metadata from each run. Below we show how to load in the RunParameters and ChannelMap information, which may be useful during analysis.

In [22]:
#all the run configuration paramenters are stored in these three files
run_parameters_file = './config/30th/Run_Parameters_Run30.xlsx'
calibrations_file = './config/30th/Calibrations_Xe_Run30.csv'
channel_map_file = './config/30th/Channel_Map_Run30.xlsx'

# Pick a dataset:
this_dataset = '20200922_Afternoon_AfterFifthInjection'

#analysis_config object loads all these paramenters
analysis_config = StruckAnalysisConfiguration.StruckAnalysisConfiguration()

analysis_config.GetRunParametersFromFile( run_parameters_file, sheet = this_dataset )
analysis_config.GetChannelMapFromFile( channel_map_file, sheet = this_dataset )




### Run parameters:

In [23]:
# RunParameters are stored as a python dict object

for parameter, value in analysis_config.run_parameters.items():
    print( '{:<25}\t{:>10.6}'.format( parameter+':', float(value) ) )

    
    
sampling_period_us = analysis_config.run_parameters['Sampling Period [ns]'] / 1.e3 # Convert from ns to us
trigger_time_samples = analysis_config.run_parameters['Pretrigger Length [samples]']
waveform_length_samples = analysis_config.run_parameters['Waveform Length [samples]']


Sampling Rate [MHz]:     	      62.5
Simulation Sampling Rate [MHz]:	    7.8125
Electrons/ADC [electrons]:	      11.9
Drift Length [mm]:       	     140.0
Drift Velocity [mm/us]:  	      1.79
Num Struck Boards:       	       3.0
Waveform Length [samples]:	   10000.0
Pretrigger Length [samples]:	    2000.0
Baseline Length [samples]:	    1000.0
Strip Threshold [sigma]: 	       5.0
Max Drift Time [us]:     	   78.2123
Sampling Period [ns]:    	      16.0
Baseline Average Time [us]:	      16.0
Energy Start Time [us]:  	     144.0
Decay Start Time [us]:   	     144.0
Decay End Time [us]:     	    1144.0
Decay Guess [us]:        	     200.0
DoInvert:                	       0.0


### Channel Map:


In [24]:
# The ChannelMap is stored as a pandas DataFramwe()
analysis_config.channel_map.head(n=32)


,Board,InputChannel,SoftwareChannel,ChannelType,ChannelName,NumDevicesInChannel,IsAmplified,IsDead,ChannelPosX,ChannelPosY,ChargeMCChannelMap
0,0,0,0,SiPM,1-1,1,False,False,-1000.0,-1000.0,None
1,0,1,1,SiPM,1-2,1,False,False,-1000.0,-1000.0,None
2,0,2,2,SiPM,1-3,1,False,False,-1000.0,-1000.0,None
3,0,3,3,SiPM,1-4,1,False,False,-1000.0,-1000.0,None
4,0,4,4,SiPM,2-1,1,False,False,-1000.0,-1000.0,None
5,0,5,5,SiPM,2-2,1,False,False,-1000.0,-1000.0,None
6,0,6,6,SiPM,2-3,1,False,False,-1000.0,-1000.0,None
7,0,8,8,SiPM,2-4,1,False,False,-1000.0,-1000.0,None
8,0,9,9,SiPM,2-5,1,False,False,-1000.0,-1000.0,None
9,0,10,10,Trigger,Trigger,1,False,False,-1000.0,-1000.0,None


<a id="accessing"></a>


# Accessing the reduced data

The reduced (a.k.a. "processed") data are stored in `pandas.DataFrame` objects, which are in turn either stored in `HDF5` or `pickle` files. In this example, we'll use data from Run29, which are stored in `HDF5` file format and can be retrieved using the `pandas.read_hdf` method.



In [25]:
#if you ls into /p/lustre1/jacopod/30th/ you can find the reconstructed data in analysis*
#where the * is for the different smoothing window, here I access 500ns as example

path_to_reduced_data = '/p/lustre1/jacopod/30th/'

#data = '/p/lustre1/jacopod/30th/20200922_Afternoon_AfterFifthInjection/analysis_500ns/'

# You can see there are several different datasets stored in this directory.
!ls $path_to_reduced_data

20200911_OvernightBi207_AfterFilling
20200912_MorningNoise_PreRecirculation
20200915_Night_AfterFirstRnInjection
20200915_Night_Noise
20200916_Morning_Noise
20200916_Night_AfterSecondRnInjection
20200916_RnPoAlphaEffTest
20200917_Night_AfterThirdRnInjection
20200920_Rn220InParallel_RnPoTest
20200921_Afternoon_AfterFourthInjection
20200922_Afternoon_AfterFifthInjection
20200923_Afternoon_AfterSixthInjection
20200924_Afternoon_AfterSeventhInjection
20201207_alpha_sim
alpha_rich_2000ns.p
alpha_rich_500ns.p
config
log


In [ ]:
# We'll load in one of the datasets from the Rn220 injection campaign. It will be stored
# in a pandas DataFrame object called "data_df".
# The reduced data fils are compiled into a pandas DataFrame for each dataset, which is written
# to disk as a pickle file (python's binary file format)

# Note: this file is pretty large, so it make take a minute or two to load.

fname = path_to_reduced_data + this_dataset + '/analysis_500ns/reduced_added.p'

import pickle

with open(fname,'rb') as input_file:
    data_df = pickle.load(input_file)
    
data_df.head()



<a id="allvariables"></a>

### Looking at all the variables in the data tree


The `.head()` method used above gives us a taste of what's contained in the data tree. However, we can look at a complete list of all the variables contained in the data by looping through all of the columns in the `DataFrame`. You can see that there is a combination of channel-level and event-level data stored in the tree, each of which can be useful for various analyses.

In [ ]:
for colname in data_df.columns:
    print(colname)

<a id="analyzing"></a>

# Analyzing the reduced data

Below we show a couple of quick plots that can be made using event-level data. Here we're using data from after the Rn220 injection, so we should expect to see some alpha events distributed throughout the TPC. Thse will show up as a peak in the Charge Energy spectrum.

### Charge energy vs. drift time:

First, let's just plot the charge energy vs. signal time.

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12


charge_energy = data_df['TotalTileEnergy']
time_of_max_channel = data_df['TimeOfMaxChannel']

#data can also be selected for example a specific energy range
mask = (data_df['TotalTileEnergy']>100) & (data_df['TotalTileEnergy']<5000)\
& (data_df['TotalSiPMEnergy']>100) & (data_df['TotalSiPMEnergy']<2000000)

#make a 2d histogram with the selected data (for example charge energy vs light energy)

xbins = np.linspace(0.,6500.,200)
ybins = np.linspace(0.,5000.,100)

#plt.hist2d(time_of_max_channel[mask], charge_energy[mask],bins=(xbins,ybins))
plt.plot(time_of_max_channel[mask], charge_energy[mask],'o',color=(0.,0.,1.,0.1),markersize=1.,markeredgecolor=(0.,0.,0.,0.))

plt.xlabel('Time of max channel (samples)')
plt.ylabel('Charge Energy (ADC units)')

Here we see a couple of features worth noting:
* There are two distinct cutoffs on the "time" axis, which correspond to the top and bottom of the detector. The trigger is at 1000 samples, so events near 1000 samples come from interactions just below the anode. Events near ~6000 samples come from the cathode (at the bottom).
* We can see a band at ~600ADC in the Charge Energy spectrum, which corresponds to the alpha signals from Bi212 decays.


We'll do two things to clean this up a bit. First, let's convert the X-axis into something a bit more meaningful -- the actual drift time.

In [ ]:
drift_time = (time_of_max_channel - analysis_config.run_parameters['Pretrigger Length [samples]']) \
               * analysis_config.run_parameters['Sampling Period [ns]'] / 1000.


#plt.hist2d(time_of_max_channel[mask], charge_energy[mask],bins=(xbins,ybins))
plt.plot(drift_time[mask], charge_energy[mask],'o',color=(0.,0.,1.,0.1),markersize=1.,markeredgecolor=(0.,0.,0.,0.))

plt.xlabel('Drift time (microseconds)')
plt.ylabel('Charge Energy (ADC units)')


Next, we can select events where the charge is only on one or two channels. This should be the case for most alpha events, since the ionization is so localized. 




In [ ]:
mask = (data_df['TotalTileEnergy']>100) & (data_df['TotalTileEnergy']<5000) \
        & (data_df['TotalSiPMEnergy']>100) & (data_df['TotalSiPMEnergy']<2000000) \
        & (data_df['NumTileChannelsHit'] < 3)

#plt.hist2d(time_of_max_channel[mask], charge_energy[mask],bins=(xbins,ybins))
plt.plot(drift_time[mask], charge_energy[mask],'o',color=(0.,0.,1.,0.1),markersize=1.,markeredgecolor=(0.,0.,0.,0.))

plt.xlabel('Drift time (microseconds)')
plt.ylabel('Charge Energy (ADC units)')


<a id="inspectingwaveforms"></a>

# Inspecting waveforms in the raw data

Sometimes it's useful to go back to the waveform-level information to better understand some feature in the reduced data. There is a class called `Event` in the `StanfordTPCAnalysis.WaveformAnalysis.Waveform` module that allows the easy plotting of all the waveforms in a given event.

For this to work, you need access to the ROOT files with the raw waveform data. For now, these are all located on the LLNL cluster, but can be moved to SLAC if needed.


In [ ]:
#the indexes of the events passing the cuts can be seen in this way 
cut_index = data_df[mask].index
this_evt = cut_index[0]

# We get the filename in which the event is stored. The filename stores the ROOT file, though, so
# I need to change the file extension to the reduced.h5 file.
this_evt_filename = data_df['File'].loc[this_evt].split('.')[0] + '_reduced.h5'
print(this_evt_filename)


#a waveform can be accessed in the following way, in this case the first event passing the cut
event = Waveform.Event(path_to_reduced_data + this_dataset + '/analysis_500ns/' + this_evt_filename,\
                       path_to_reduced_data + this_dataset + '/raw_data/',\
                       this_evt,\
                       run_parameters_file,\
                       calibrations_file,\
                       channel_map_file)

#the second parameter in the Event class is the location of the tier1 file, which is where the low level information
#are stored, such as the waveform. Make sure the dataset (in this case 20200213_AfterRnInjection matches with the one
#opened in the reduced_added.h5 file), the third parameter is the number of the event you want to see the waveform of

In [ ]:
smoothing_windows_us = 0.5
#the waveform can be smoothed (just for visualization)
event.smooth(smoothing_windows_us)
plot = event.plot_event(risetime=True)
plot.show()